In [5]:
import backtrader as bt
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt 
import numpy as np
from finlab import crypto
import yfinance as yf
import matplotlib.dates 
import warnings

In [8]:
class SmaCross(bt.Strategy):
    params = dict(
        exitbars=5,
        pfast=30,  # period for the fast moving average
        pslow=60   # period for the slow moving average
    )
    
    def __init__(self):
        self.dataclose = self.datas[0].close
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal
        self.order = None
    
    
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log("Buy Executed {}".format(order.executed.price))
            elif order.issell():
                self.log("Sell Executed {}".format(order.executed.price))
            
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log("Order Canceled/Margin/Rejected")

        self.order = None


    def next(self):
        self.log("Close {}".format(self.dataclose[0]))
        if self.order:
            return
        
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.log("Buy Create {}".format(self.dataclose[0]))
                self.order = self.buy()

        elif self.crossover < 0:  # in the market & cross to the downside
            self.log("Sell Create {}".format(self.dataclose[0]))
            self.order = self.close()
        
        else: # 持續持有10天就賣出
            if len(self) >= (self.bar_executed + self.params.exitbars):
                self.log("Sell Create {}".format(self.dataclose[0]))
                self.order = self.sell()

    # def next(self):
    #     self.log("Close {}".format(self.dataclose[0]))
    #     if self.order:
    #         return

    #     if not self.position:
    #         if self.dataclose[0] < self.dataclose[-1]:
    #             if self.dataclose[-1] < self.dataclose[-2]:
    #                 self.log("Buy Create {}".format(self.dataclose[0]))
    #                 self.order = self.buy()
    #     else:
    #         if len(self) >= (self.bar_executed + self.params.exitbars):
    #             self.log("Sell Create {}".format(self.dataclose[0]))
    #             self.order = self.sell()

    def log(self, txt):
        time = self.datas[0].datetime.date(0)
        # print("{} {}".format(time.isoformat(), txt))

    def stop(self):
        print(self.params.exitbars, self.params.pfast, self.params.pslow, self.broker.getvalue())

In [9]:
from backtrader import cerebro


if __name__ =='__main__':
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(10000)   #初始資金
    cerebro.broker.setcommission(commission = 0.001)    #手續費 0.1%

    # symbol = "SPY"
    # date = "2020-01-01"
    # data = yf.download(symbol,date)
   
    data = bt.feeds.YahooFinanceCSVData(dataname = "data/SPY.csv") #,fromdate = dt(2021,1,1), todate = dt(2022,10,1)


    #df = df[['Open', 'High', 'Low', 'Close', 'Volume']]

    #df = crypto.get_all_binance('BTCUSDT', '4h')
    #df['Datetime'] = pd.to_datetime(df[0])
    #df.set_index('Datetime')
    #df_BTC = bt.feeds.PandasData(df, fromdate = dt.datetime(2020,1,1))

    cerebro.adddata(data)
    cerebro.addstrategy(SmaCross)
    # cerebro.optstrategy(SmaCross, 
    #                     exitbars = 5, 
    #                     pfast = range(5, 30, 5), 
    #                     pslow = range(60, 120, 10) 
    #                     )
    print("Start Portfolio {}" .format(cerebro.broker.getvalue()))
    cerebro.run()
    print("Final Portfolio {}" .format(cerebro.broker.getvalue()))

Start Portfolio 10000
5 30 60 9992.22534
Final Portfolio 9992.22534
